# Before you start

To run this notebook you first have to:
1. Install the `progressreporting` package following the instructions [here](https://github.com/SengerM/progressreporting).
2. [Create a Telegram bot](https://core.telegram.org/bots#3-how-do-i-create-a-bot) and have in hand its token and also the ID of the chat to which you want to send the messages (basic tutorial [here](https://github.com/SengerM/progressreporting#creating-a-bot), otherwise just google it).

# Normal execution with no errors

Import everything:

In [ ]:
from progressreporting.TelegramProgressReporter import TelegramProgressReporter
from data_processing_bureaucrat.Bureaucrat import TelegramReportingInformation # Here I store the token of the bot, NEVER make it public!
import time

MAX_K = 999
SLEEP_PER_LOOP = 3e-2 # Seconds.
REPORTING_TIME = 5 # Seconds.

You will not need the `TelegramReportingInformation`, this is just a private repo I have to store both the token and the chat ID of my bot. **Never make the token public to the world!** If you make it public, it will not be long until the world stoles your bot (already happened to me).

In [ ]:
with TelegramProgressReporter(
    total = MAX_K, # Tell me how long is the loop.
    telegram_token = TelegramReportingInformation().token, # Never make public the token!
    telegram_chat_id = TelegramReportingInformation().chat_id, # Tell me to which chat send the messages.
    miminum_update_time_seconds = REPORTING_TIME, # Default is 60 seconds to not affect performance.
    loop_name = 'PyHEP 2021 - Loop without errors', # This is optional.
) as reporter:
    for k in range(MAX_K):
        # Do some stuff...
        time.sleep(SLEEP_PER_LOOP)
        reporter.update(1) # 1 means 1/MAX_K completed. After REPORTING_TIME it will send a message.

# An error during the loop

If something happens, you will be notified:

In [ ]:
with TelegramProgressReporter(total = MAX_K, telegram_token = TelegramReportingInformation().token, telegram_chat_id = TelegramReportingInformation().chat_id, miminum_update_time_seconds = REPORTING_TIME, 
    loop_name = 'PyHEP 2021 - Loop with an error',
) as reporter:
    for k in range(MAX_K):
        # Do some stuff...
        time.sleep(SLEEP_PER_LOOP)
        if k > 333:
            raise RuntimeError('Unexpected things can happen!')
        reporter.update(1)

# Multiple reporters at the same time

You can use the same bot and chat to report from many loops at the same time. The loops can be happening in the same computer, or in diferent computers (e.g. one at the lab and another at you office).

To ilustrate I will run 3 dummy loops using the `multiprocessing` package.

In [ ]:
from multiprocessing import Pool

In [ ]:
def function_with_reporting_loop(loop_number: int):
    # This function is defined just to be able to 
    # run many loops at the same time using the 
    # multiprocessing package.
    time.sleep(loop_number) # Wait some time to avoid all the loops to start at the same time.
    with TelegramProgressReporter(total = MAX_K, telegram_token = TelegramReportingInformation().token, telegram_chat_id = TelegramReportingInformation().chat_id, miminum_update_time_seconds = REPORTING_TIME, 
        loop_name = f'Loop number {loop_number}',
    ) as reporter:
        for k in range(MAX_K):
            # Do some stuff...
            time.sleep(SLEEP_PER_LOOP)
            reporter.update(1)

The function `function_with_reporting_loop` is just needed for the `multiprocessing` package. Usually each instance of `TelegramProgressReporter` is runing in different scripts/different computers.

Now we just run 3 instances simultaneously:

In [ ]:
with Pool(3) as p:
    p.map(function_with_reporting_loop, [1, 2, 3])